In [1]:
#1) shen atlas stuff
#2) set up loop code for 1 story (e.g. pieman run1)
#- need fmri
#- need confounds

#3) extract ts
#4) do isc for auditory cortex




## import modules

In [2]:
import glob 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sys, os, nilearn
from nilearn import input_data, datasets
from nilearn import datasets, plotting
from nilearn.image import new_img_like, load_img, get_data

import nest_asyncio
nest_asyncio.apply()


import datalad
from datalad.api import install

/opt/anaconda3/envs/neuroimaging/lib/python3.9/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


### set up masker

In [3]:
shen_dictionary = np.load('/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/narratives_pieman/data_pieman//shen_dictionary.npy', allow_pickle = True).item()
shen_file       = '/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/narratives_pieman/data_pieman//shen_2mm_268_parcellation.nii'

path              = "/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/"

target_dir        = '/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/narratives_pieman/data_pieman/extract_ts/'
prepro_data_path  = '/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/'
relevant_columns  = ['csf','white_matter','trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y','rot_z']



#Shen
shen_masker = input_data.NiftiLabelsMasker(shen_file, 
                                             detrend     = True, 
                                             standardize = True,
                                             high_pass   = 0.01, 
                                             t_r         = 1.5, 
                                             low_pass    = 0.12 );

#Pauli
subcortex = datasets.fetch_atlas_pauli_2017()
pauli_masker = input_data.NiftiMapsMasker(subcortex.maps, 
                                             detrend     = True, 
                                             standardize = True,
                                             high_pass   = 0.01, 
                                             t_r         = 1.5, 
                                             low_pass    = 0.12 );



#AAN
aan = '/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/narratives_pieman/data_pieman/AAN_MNI152_1mm_v1p0/AAN_Combined_Labels.nii'
aan_masker = input_data.NiftiMapsMasker(   aan, 
                                             detrend     = True, 
                                             standardize = True,
                                             high_pass   = 0.01, 
                                             t_r         = 1.5, 
                                             low_pass    = 0.12 );


## set up files

In [6]:
task_list = [   '-pieman_space-MNI152NLin2009cAsym',             #  68
                # The Moth
                # 7:30 
            ]           


for curr_task in task_list[:]:
    print(curr_task)
    search_path = path + "/**/*" + str(curr_task) + "_res-native_desc-preproc_bold.nii.gz"
    #print(search_path)
    text_files = glob.glob(search_path, recursive = True)
    #print(text_files[:1])
    print(len(text_files))
    

    subjs = []
    for i in range(len(text_files)):
        subjs.append(text_files[i][82:89])

    print(subjs)
    
    
    
    ## prepare extraction
    file_appendix     = '_task' + str(curr_task) + '_LTR_Z_THP001_LP012_shen268_pauli16_aan9.npy'
    #print(file_appendix)
    
    bold_file_ending  = '_task' + str(curr_task) + '_res-native_desc-preproc_bold.nii.gz'
    #print(bold_file_ending)
    
    conf_file_ending  = '_task' +str (curr_task[:-25]) +   'desc-confounds_regressors.tsv'
    #print(conf_file_ending)

    ## loop extraction
    for curr_subj in subjs[:]:
        sys.stdout.write(str(curr_subj) + ',')
        
        outname            = target_dir + curr_subj + file_appendix

        if not os.path.exists(outname):
        
            func_filename      = prepro_data_path + curr_subj + '/func/' + curr_subj + bold_file_ending;
            conf_filename      = prepro_data_path + curr_subj + '/func/' + curr_subj + conf_file_ending;

            datalad.api.get(func_filename) 
            datalad.api.get(conf_filename) 

            confounds_data = pd.read_csv(conf_filename, sep ='\t')
            confounds_data = confounds_data[relevant_columns].values

            outname            = target_dir + curr_subj + file_appendix

            if not os.path.exists(outname):
                time_series1 = shen_masker.fit_transform(func_filename, confounds=confounds_data)
                time_series2 = pauli_masker.fit_transform(func_filename, confounds=confounds_data)
                time_series3 = aan_masker.fit_transform(func_filename, confounds=confounds_data)

                time_series = np.hstack([time_series1, time_series2, time_series3])



                np.save(outname, time_series) 

        # drop stuff - see if that works ...
        !datalad drop -d '/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/' -r

-pieman_space-MNI152NLin2009cAsym
68
['sub-021', 'sub-026', 'sub-019', 'sub-017', 'sub-028', 'sub-043', 'sub-044', 'sub-072', 'sub-075', 'sub-081', 'sub-029', 'sub-027', 'sub-018', 'sub-020', 'sub-080', 'sub-074', 'sub-073', 'sub-045', 'sub-042', 'sub-060', 'sub-067', 'sub-058', 'sub-051', 'sub-056', 'sub-069', 'sub-033', 'sub-034', 'sub-057', 'sub-068', 'sub-050', 'sub-066', 'sub-059', 'sub-061', 'sub-035', 'sub-032', 'sub-078', 'sub-047', 'sub-040', 'sub-049', 'sub-076', 'sub-082', 'sub-071', 'sub-025', 'sub-022', 'sub-070', 'sub-048', 'sub-077', 'sub-041', 'sub-079', 'sub-046', 'sub-023', 'sub-024', 'sub-039', 'sub-037', 'sub-030', 'sub-064', 'sub-063', 'sub-055', 'sub-052', 'sub-031', 'sub-009', 'sub-036', 'sub-038', 'sub-007', 'sub-053', 'sub-054', 'sub-062', 'sub-065']
action summary:                                                                 
  drop (notneeded: 50625)
action summary:                                                                 
  drop (notneeded: 50625)


action summary:
  drop (notneeded: 50623, ok: 2)
drop(ok): /Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/sub-041/func/sub-041_task-pieman_desc-confounds_regressors.tsv (file) [checking fcp-indi...]
drop(ok): /Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/sub-041/func/sub-041_task-pieman_space-MNI152NLin2009cAsym_res-native_desc-preproc_bold.nii.gz (file) [checking fcp-indi...]
action summary:
  drop (notneeded: 50623, ok: 2)
drop(ok): /Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/sub-079/func/sub-079_task-pieman_desc-confounds_regressors.tsv (file) [checking fcp-indi...]
drop(ok): /Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/sub-079/func/sub-079_task-pieman_space-MNI152NLin2009cAsym_res-native_desc-preproc_bold.nii.gz (file) [checking fcp-indi...]
action summary:
  drop (notneeded: 50623, ok: 2)
drop(ok): /Users/ralfschmaelzle/Documents/GITHUB/

action summary:
  drop (notneeded: 50623, ok: 2)
drop(ok): /Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/sub-007/func/sub-007_task-pieman_desc-confounds_regressors.tsv (file) [checking fcp-indi...]
drop(ok): /Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/sub-007/func/sub-007_task-pieman_space-MNI152NLin2009cAsym_res-native_desc-preproc_bold.nii.gz (file) [checking fcp-indi...]
action summary:
  drop (notneeded: 50623, ok: 2)
drop(ok): /Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/sub-053/func/sub-053_task-pieman_desc-confounds_regressors.tsv (file) [checking fcp-indi...]
drop(ok): /Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/sub-053/func/sub-053_task-pieman_space-MNI152NLin2009cAsym_res-native_desc-preproc_bold.nii.gz (file) [checking fcp-indi...]
action summary:
  drop (notneeded: 50623, ok: 2)
drop(ok): /Users/ralfschmaelzle/Documents/GITHUB/

In [7]:
!datalad drop -d '/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/NARRATIVES/fmriprep/' -r

action summary:                                                                 
  drop (notneeded: 50625)
